Explore psychological flexibility and engage EMA data after preprocessing (see other NBs for preprocessing code)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [ ]:
path_PF_clean = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/pf_final.csv'

In [ ]:
#read in csv from preprocessed psychologial felxibility EMAs 
data_PF = pd.read_csv(path_PF_clean)
data_PF = pd.DataFrame(data = data_PF)

print('Original data_PF_S3 shape:\n', data_PF.shape, '\n')
#ensure no replicate ID (211 participants in study)
print('Original data_PF_S3 unique IDs:\n', data_PF['participant_id'].unique().shape, '\n')
#how much missing data is there?
print('Original data_PF_S3 missing value counts:\n', data_PF.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data_PF_S3 data types:\n', data_PF.info(), '\n')
#what is the participant response rate across the entire study?
print('Non-response rate for PF survey:\n', data_PF['completed_ts_utc'].isnull().sum() / data_PF.shape[0] * 100, '%')
#add a binary column for if survey was completed (1) or not (0)
data_PF['completed'] = np.where(data_PF['results_updated'].isnull(), 0, 1)

In [ ]:
#make new column corresponding to the number in series of survey sent for each participant
part_id = data_PF['participant_id'].unique()
for participant in part_id:
    surveys = data_PF[data_PF['participant_id'] == participant]
    data_PF.loc[data_PF['participant_id'] == participant, 'survey_count'] = np.arange(surveys.shape[0])

In [ ]:
#change dates from objects to datetimes
data_PF['survey_dt'] = data_PF['survey_dt'].astype('datetime64')
data_PF['delivered_ts_utc'] = data_PF['delivered_ts_utc'].astype('datetime64[ns]')
data_PF['started_ts_utc'] = data_PF['started_ts_utc'].astype('datetime64[ns]')
data_PF['completed_ts_utc'] = data_PF['completed_ts_utc'].astype('datetime64[ns]')
data_PF['ingested_ts_utc'] = data_PF['ingested_ts_utc'].astype('datetime64[ns]')

#confirm change 
print(data_PF.info())

In [ ]:
#activity column contains str and float but right now dtype is object, update accordingly
for index, row in data_PF_completed.iterrows():
    try:
        row['activity'] = float(row['activity'])
    except:
        pass

In [ ]:
#compute time between when survey is sent and when participant starts the survey
#data_PF['start_delay_dt'] = data_PF['started_ts_utc'] - data_PF['delivered_ts_utc']
data_PF['start_delay'] = (data_PF['started_ts_utc'] - data_PF['delivered_ts_utc']).astype('timedelta64[m]')
#compute time between when survey is sent and when participant starts the survey
data_PF['time_to_complete'] = (data_PF['completed_ts_utc'] - data_PF['started_ts_utc']).astype('timedelta64[m]')

In [ ]:
#each participant should have 50 survey entries
data_PF['participant_id'].value_counts()

In [ ]:
#all participants should have a survey_count of 49 (sent 50 surveys)
data_PF.groupby('participant_id')['survey_count'].max().sort_values()

In [ ]:
#split off data for surveys that were completed
data_PF_completed = data_PF[data_PF['completed'] == 1]
#null survey questions for surveys that were completed
data_PF_completed.isnull().sum()

In [ ]:
#there should be no nans in columns 'activity' and 'pf_3' to 'pf_mgt', find these rows
pf_no_comp = data_PF_completed.loc[data_PF_completed.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1) > 0]
print(pf_no_comp.shape)
pf_no_comp.to_csv('pf_no_comp.csv')

In [ ]:
#there should be at least one entry for columns 'exp_0' to 'exp_13', find rows that do not have at least one entry
exp_no_comp = data_PF_completed.loc[data_PF_completed.loc[:, 'exp_0':'exp_13'].isnull().sum(axis=1) > 13]
print(exp_no_comp.shape)
exp_no_comp.to_csv('exp_no_comp.csv')

In [ ]:
#number of participants sent surveys across time
ax = data_PF.groupby('survey_count')['participant_id'].count().plot()
#number of participants completed surveys across time
ax = data_PF[data_PF['completed'] == 1].groupby('survey_count')['completed'].count().plot()
plt.legend(('sent', 'completed'))
plt.title('Surveys sent vs completed across time')
ax.set_ylabel('Count')
plt.show

In [ ]:
#number of participants sent surveys across time
ax = data_PF.groupby('survey_dt')['participant_id'].count().plot()
#number of participants completed surveys across time
ax = data_PF[data_PF['completed'] == 1].groupby('survey_dt')['completed'].count().plot()
plt.legend(('sent', 'completed'))
plt.title('Surveys sent vs completed across time')
ax.set_ylabel('Count')
plt.show

In [ ]:
ax = (data_PF[data_PF['completed'] == 1].groupby('survey_count')['participant_id'].count() /
data_PF.groupby('survey_count')['participant_id'].count()*100).plot()

ax.set_ylabel('Response rate (%)')
plt.title('Survey response rate across time')
plt.show

In [ ]:
#delay to starting survey across survey dates
ax = data_PF_completed.groupby('survey_count')['start_delay'].mean().plot()

plt.title('Delay to starting survey across survey dates')
ax.set_ylabel('Min')
plt.show

In [ ]:
#time to complete survey across survey dates
ax = data_PF_completed.groupby('survey_count')['time_to_complete'].mean().plot()

plt.title('Time to complete survey across survey dates')
ax.set_ylabel('Min')
plt.show

In [ ]:
#time to complete survey across participants
ax = data_PF_completed.groupby(['participant_id'])['time_to_complete'].mean().plot(kind='hist', bins = 75)

plt.title('Time to complete survey across participants')
ax.set_ylabel('Min')
plt.show

In [ ]:
#pf_mgt across survey dates
ax = data_PF_completed.groupby(['survey_count'])['pf_mgt'].mean().plot()

plt.title('pf_mgt across survey dates')
ax.set_ylabel('Min')
plt.show

In [ ]:
dep_vars = ['pf_mgt', 'exp_neg', 'exp_pos', 'exp_neut', 'survey_count', 'start_delay', 'time_to_complete']
sns.pairplot(data_PF_completed[dep_vars].dropna(), kind='reg')

In [ ]:
data_PF_completed[dep_vars].corr()

In [ ]:
data_PF_completed.head(100)